In [1]:
import boto3
import json
from botocore.exceptions import ClientError

In [3]:
#session = boto3.Session(profile_name='workshop')
#config = session.client('config', region_name='us-west-2')
#s3_client = session.client('s3')

In [10]:
config = boto3.client('config')
s3_client = boto3.client('s3')

In [20]:
def create_custom_rule(rule_name, rule_description, resource_type, rule_content, input_parameters=None, enable_debug=False, tags=None):
    rule = {
        'ConfigRuleName': rule_name,
        'Description': rule_description,
        'Scope': {
            'ComplianceResourceTypes': [
                resource_type,
            ],
        },
        'Source': {
            'Owner': 'CUSTOM_POLICY',
            'SourceDetails': [
                {
                    'EventSource': 'aws.config',
                    'MessageType': 'ConfigurationItemChangeNotification'
                },
            ],
            'CustomPolicyDetails': {
                'PolicyRuntime': 'guard-2.x.x',
                'PolicyText': rule_content,
                'EnableDebugLogDelivery': enable_debug,
            },
        }
    }
    if input_parameters is not None:
        rule['InputParameters'] = input_parameters
    if tags is None:
        tags = []
    try:
        response = config.put_config_rule(ConfigRule=rule, Tags=tags)
        print('Successfully created Config rule: ' + json.dumps(response, indent=2, default=str))
    except ClientError as e:
        print('Error creating Config rule: ' + e.response['Error']['Code'])
    except Exception as e:
        print('Error creating Config rule: ' + str(e))

In [7]:
rule_name = 'dynamodb-pitr-enabled'
rule_description = 'Checks if DynamoDB table has Point-in-time Recovery enabled'
resource_type = 'AWS::DynamoDB::Table'
rule_content = """let status = ['ACTIVE']

rule tableisactive when
    resourceType == "AWS::DynamoDB::Table" {
    configuration.tableStatus == %status
}

rule checkcompliance when
    resourceType == "AWS::DynamoDB::Table"
    tableisactive {
        let pitr = supplementaryConfiguration.ContinuousBackupsDescription.pointInTimeRecoveryDescription.pointInTimeRecoveryStatus
        %pitr == "ENABLED"
}"""
create_custom_rule(rule_name, rule_description, resource_type, rule_content, enable_debug=True)


Successfully created Config rule: {
  "ResponseMetadata": {
    "RequestId": "8488a7da-69da-401e-877e-f49a233794c1",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "8488a7da-69da-401e-877e-f49a233794c1",
      "strict-transport-security": "max-age=86400",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "0",
      "date": "Wed, 31 May 2023 12:33:04 GMT"
    },
    "RetryAttempts": 0
  }
}


In [22]:
rule_name = 's3-bucket-versioning-enabled-test-new'
rule_description = 'Checks that S3 buckets have versioning enabled'
resource_type = 'AWS::S3::Bucket'
rule_content = """rule check_bucketversioning {
     supplementaryConfiguration.BucketVersioningConfiguration.status == "Enabled" <<
     result: NON_COMPLIANT
     message: S3 Bucket Versioning is NOT enabled.
     >>
}
"""
create_custom_rule(rule_name, rule_description, resource_type, rule_content)

Successfully created Config rule: {
  "ResponseMetadata": {
    "RequestId": "773f412a-7bfb-4436-9ef7-ed619b85927d",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "773f412a-7bfb-4436-9ef7-ed619b85927d",
      "strict-transport-security": "max-age=86400",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "0",
      "date": "Wed, 31 May 2023 12:58:51 GMT"
    },
    "RetryAttempts": 0
  }
}


In [21]:
rule_name = 'ec2-eip-running-test-new'
rule_description = 'Checks that EC2 EIPs are associated with running instances'
resource_type = 'AWS::EC2::Instance'
input_parameters = "{\"volumeType\":\"gp3\"}"
tags = [{'Key': 'Config', 'Value': 'customRule'}]
rule_content = """let eipresource = relationships.*[ resourceType  == 'AWS::EC2::EIP' ]


rule check_ec2_eip_compliance {
    when %eipresource !empty {
    configuration.state.name == "running"
}
}
"""
create_custom_rule(rule_name, rule_description, resource_type, rule_content, input_parameters, tags=tags)

Successfully created Config rule: {
  "ResponseMetadata": {
    "RequestId": "ffad402e-8e54-4a86-b9f5-7b87f89d93fb",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "x-amzn-requestid": "ffad402e-8e54-4a86-b9f5-7b87f89d93fb",
      "strict-transport-security": "max-age=86400",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "0",
      "date": "Wed, 31 May 2023 12:53:54 GMT"
    },
    "RetryAttempts": 0
  }
}


### Conformance packs

In [11]:
# Upload a file to S3 bucket
file = 'conf_pack.yaml'
bucket_name = 'config-bucket-089715336747'
s3_client.upload_file(file, bucket_name, file)

In [13]:
# Create a sns topic and get the arn
sns_client = boto3.client('sns')
response = sns_client.create_topic(Name='config-topic')
topic_arn = response['TopicArn']

In [15]:
# Create conformance pack
response = config.put_conformance_pack(
    ConformancePackName='conformance-pack-1',
    TemplateS3Uri=f's3://{bucket_name}/{file}',
    DeliveryS3Bucket=bucket_name,
    TemplateBody='string',
    ConformancePackInputParameters=[
        {
            'ParameterName': 'SnsTopicForPublishNotificationArn',
            'ParameterValue': topic_arn
        },
    ]
)
print(response)

{'ConformancePackArn': 'arn:aws:config:us-east-1:089715336747:conformance-pack/conformance-pack-1/conformance-pack-8myse008n', 'ResponseMetadata': {'RequestId': '024810b6-2fc5-49b3-bb53-7718cb1cd8b2', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '024810b6-2fc5-49b3-bb53-7718cb1cd8b2', 'strict-transport-security': 'max-age=86400', 'content-type': 'application/x-amz-json-1.1', 'content-length': '125', 'date': 'Wed, 31 May 2023 12:36:38 GMT'}, 'RetryAttempts': 0}}
